In [1]:
#- 패키지 임포트
import pandas as pd
import numpy as np
#import emoji # 이모지 제거용
import re
from soynlp.normalizer import * #반복되는 자음 제거
import time
from tqdm import tqdm #for문 진행률 
from time import sleep #for문 진행 시간 
from kiwipiepy import Kiwi #문장 분리2

from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)

Output()

In [2]:
''' 파일 불러오기 '''
df = pd.read_csv("../dataset/2022/본문_크롤링/나무문화_본문_2022_5월.txt", sep='\t', encoding='UTF-8')
# print(df.shape, type(df)) # 파일 구조 및 컬럼명 확인용
df['org_idx']= df.index # 인덱스 column 만들기: 문장 분리할 때 사용함
# print(df.columns) # org_idx 제대로 추가됐나 단순 확인용.
post_list = list(df['본문']) #자연어 전처리를 위해 본문을 리스트 형태로

In [3]:
''' 불용어 제거 '''
# post_list 양이 많을 경우 불용어 제거 중에 오류 나면 다시 리스트로 만드는 데 오래 걸려서 카피하는 것임. 안 하고 post_list로 진행해도 됨.
post_list2 = post_list.copy()
for i in range(len(post_list2)):
    post_list2[i] = re.sub('\n', ' ', string=str(post_list2[i])) # 줄바꿈을 띄어쓰기 하나로
    post_list2[i] = re.compile('[^|ㄱ-ㅎ|ㅏ-ㅣ|A-Z|a-z|0-9|가-힣]+').sub(' ',post_list2[i])
    post_list2[i] = repeat_normalize(post_list2[i], num_repeats=1)
	# ↑ ㅋㅋㅋ, ㅎㅎㅎ 등의 의미 없는 자음 삭제. num_repeats가 1이면 반복 글자 중 1개만 남기고 2면 2개만 남김.
    post_list2[i] = re.sub(r"^\s+|\s+$", "", post_list2[i]) # 문서 앞뒤 공백 제거
    post_list2[i] = post_list2[i].split() # 문서 내 공백(1개 이상) 기준으로 자르기
    post_list2[i] = ' '.join(post_list2[i]) # 공백 기준으로 나뉜 문서를 다시 1개의 공백을 두고 붙임.
	# ↑ 문서 내 다중 공백을 지우기 위함.]

df['cleaning'] = pd.DataFrame(post_list2) #불용어 제거된 본문을 데이터 프레임으로

In [4]:
''' 문장 분리 '''
kiwi = Kiwi() #kss보다 속도 빠름 
split_list = [kiwi.split_into_sents(df['cleaning'][i]) for i in tqdm(range(len(df['cleaning'])))]
df['split_list'] = split_list
df

100%|██████████| 19231/19231 [37:39<00:00,  8.51it/s]  


,url,title,date,본문,org_idx,cleaning,split_list
0,https://blog.naver.com/swallow1210/222754267899,(31) 전쟁 역사의 원형이 남아 있는 펠로폰네소스: 그리스(4),2022.05.31.,아테네 이외에도 그리스는 국토 전역이 문화유산이어서 어느 지역을 가더라도 고대문화의...,0,아테네 이외에도 그리스는 국토 전역이 문화유산이어서 어느 지역을 가더라도 고대문화의...,[(아테네 이외에도 그리스는 국토 전역이 문화유산이어서 어느 지역을 가더라도 고대문...
1,https://blog.naver.com/gowk83/222754266889,ICT 생활 문화센터 프로그램 후기/2022 화성 오디세이 전시회,2022.05.31.,로얄엑스 데이지\n&\n에이씨티 그라운드\nICT 생활문화센터\n2022.05.15...,1,로얄엑스 데이지 에이씨티 그라운드 ICT 생활문화센터 2022 05 15 일 마감하...,[(로얄엑스 데이지 에이씨티 그라운드 ICT 생활문화센터 2022 05 15 일 마...
2,https://blog.naver.com/mistaeri/222754267378,비행기에서 쓴 비밀쪽지 #그린애플 출판사 임정진,2022.05.31.,"제 11회 #열린아공문학상 수상작\n각각 미국, 프랑스 등으로 입양간\n어른이 된 ...",2,제 11회 열린아공문학상 수상작 각각 미국 프랑스 등으로 입양간 어른이 된 한국 아...,[(제 11회 열린아공문학상 수상작 각각 미국 프랑스 등으로 입양간 어른이 된 한국...
3,https://blog.naver.com/alway_for_u/222754267121,2022년 5월 봄과 여름 사이 ☘️,2022.05.31.,4월부터 열심히 필라테스 하는중💪\n그룹레슨임에도 기구나 환경은 예전에 이용하던 센...,3,4월부터 열심히 필라테스 하는중 그룹레슨임에도 기구나 환경은 예전에 이용하던 센터보...,[(4월부터 열심히 필라테스 하는중 그룹레슨임에도 기구나 환경은 예전에 이용하던 센...
4,https://blog.naver.com/aprilhanna/222754265618,"[광주문화예술회관]6월 시원한, 광주 공연을 소개합니다.",2022.05.31.,"안녕하세요.\n단단하고 감사하는 ""최고구마"" 입니다.\n오늘은 곧 다가오는 6월 광...",4,안녕하세요 단단하고 감사하는 최고구마 입니다 오늘은 곧 다가오는 6월 광주문화예술회...,"[(안녕하세요 단단하고 감사하는 최고구마 입니다, 0, 24, None, []), ..."
...,...,...,...,...,...,...,...
19226,https://blog.naver.com/pencilvase/222716871213,류인서 시인의 좋은시 '어둠의 단애'_시집 <그는 늘 왼쪽에...,2022.05.01.,"© Xosean, 출처 Pixabay\n어둠의 단애\n류인서\n저문다는 것, 날 저...",19226,Xosean 출처 Pixabay 어둠의 단애 류인서 저문다는 것 날 저문다는 것은 ...,[(Xosean 출처 Pixabay 어둠의 단애 류인서 저문다는 것 날 저문다는 것...
19227,https://blog.naver.com/bitnahanyu/222716870322,쏜살같이 지나간 4월,2022.05.01.,쏜살같이 지나간 4월을 정리해보려 한다.\n+ 내일 5.1 근로자의날 일요일인거 킹...,19227,쏜살같이 지나간 4월을 정리해보려 한다 내일 5 1 근로자의날 일요일인거 킹받음 4...,[(쏜살같이 지나간 4월을 정리해보려 한다 내일 5 1 근로자의날 일요일인거 킹받음...
19228,https://blog.naver.com/mohastudy/222707336028,<원리한자 8자 천자문> 81. 광원면막 암수묘명 - 曠遠綿邈하고...,2022.05.01.,안녕하세요!\n좋은인연 모하입니다.\n오늘은\n8자씩 공부하는\n천자문\n81 번째...,19228,안녕하세요 좋은인연 모하입니다 오늘은 8자씩 공부하는 천자문 81 번째 시간입니다 ...,"[(안녕하세요 좋은인연 모하입니다, 0, 16, None, []), (오늘은 8자씩..."
19229,https://blog.naver.com/woaishan89/222693417548,[서울 이벤트]남산 벚꽃길 걷기 챌린지 도전해서 N서울타워...,2022.05.01.,안녕하세요!\n다이어트매니아 '인천이찐이야'입니다.\n'워크온'이라는 어플은 걸으면...,19229,안녕하세요 다이어트매니아 인천이찐이야 입니다 워크온 이라는 어플은 걸으면서 몇보를 ...,"[(안녕하세요 다이어트매니아 인천이찐이야 입니다, 0, 24, None, []), ..."


In [5]:
import pickle
# 데이터프레임을 피클 파일로 저장함으로써 데이터타입 보존
with open('5월_문장분리.pkl', 'wb') as file:
    pickle.dump(df, file)

In [6]:
import pickle
# 저장된 피클 파일을 불러오기 익일 재실행 경우, 이 코드만 돌리면 시간절약 가능.
with open('5월_문장분리.pkl', 'rb') as file:
    df = pickle.load(file)
df

,url,title,date,본문,org_idx,cleaning,split_list
0,https://blog.naver.com/swallow1210/222754267899,(31) 전쟁 역사의 원형이 남아 있는 펠로폰네소스: 그리스(4),2022.05.31.,아테네 이외에도 그리스는 국토 전역이 문화유산이어서 어느 지역을 가더라도 고대문화의...,0,아테네 이외에도 그리스는 국토 전역이 문화유산이어서 어느 지역을 가더라도 고대문화의...,[(아테네 이외에도 그리스는 국토 전역이 문화유산이어서 어느 지역을 가더라도 고대문...
1,https://blog.naver.com/gowk83/222754266889,ICT 생활 문화센터 프로그램 후기/2022 화성 오디세이 전시회,2022.05.31.,로얄엑스 데이지\n&\n에이씨티 그라운드\nICT 생활문화센터\n2022.05.15...,1,로얄엑스 데이지 에이씨티 그라운드 ICT 생활문화센터 2022 05 15 일 마감하...,[(로얄엑스 데이지 에이씨티 그라운드 ICT 생활문화센터 2022 05 15 일 마...
2,https://blog.naver.com/mistaeri/222754267378,비행기에서 쓴 비밀쪽지 #그린애플 출판사 임정진,2022.05.31.,"제 11회 #열린아공문학상 수상작\n각각 미국, 프랑스 등으로 입양간\n어른이 된 ...",2,제 11회 열린아공문학상 수상작 각각 미국 프랑스 등으로 입양간 어른이 된 한국 아...,[(제 11회 열린아공문학상 수상작 각각 미국 프랑스 등으로 입양간 어른이 된 한국...
3,https://blog.naver.com/alway_for_u/222754267121,2022년 5월 봄과 여름 사이 ☘️,2022.05.31.,4월부터 열심히 필라테스 하는중💪\n그룹레슨임에도 기구나 환경은 예전에 이용하던 센...,3,4월부터 열심히 필라테스 하는중 그룹레슨임에도 기구나 환경은 예전에 이용하던 센터보...,[(4월부터 열심히 필라테스 하는중 그룹레슨임에도 기구나 환경은 예전에 이용하던 센...
4,https://blog.naver.com/aprilhanna/222754265618,"[광주문화예술회관]6월 시원한, 광주 공연을 소개합니다.",2022.05.31.,"안녕하세요.\n단단하고 감사하는 ""최고구마"" 입니다.\n오늘은 곧 다가오는 6월 광...",4,안녕하세요 단단하고 감사하는 최고구마 입니다 오늘은 곧 다가오는 6월 광주문화예술회...,"[(안녕하세요 단단하고 감사하는 최고구마 입니다, 0, 24, None, []), ..."
...,...,...,...,...,...,...,...
19226,https://blog.naver.com/pencilvase/222716871213,류인서 시인의 좋은시 '어둠의 단애'_시집 <그는 늘 왼쪽에...,2022.05.01.,"© Xosean, 출처 Pixabay\n어둠의 단애\n류인서\n저문다는 것, 날 저...",19226,Xosean 출처 Pixabay 어둠의 단애 류인서 저문다는 것 날 저문다는 것은 ...,[(Xosean 출처 Pixabay 어둠의 단애 류인서 저문다는 것 날 저문다는 것...
19227,https://blog.naver.com/bitnahanyu/222716870322,쏜살같이 지나간 4월,2022.05.01.,쏜살같이 지나간 4월을 정리해보려 한다.\n+ 내일 5.1 근로자의날 일요일인거 킹...,19227,쏜살같이 지나간 4월을 정리해보려 한다 내일 5 1 근로자의날 일요일인거 킹받음 4...,[(쏜살같이 지나간 4월을 정리해보려 한다 내일 5 1 근로자의날 일요일인거 킹받음...
19228,https://blog.naver.com/mohastudy/222707336028,<원리한자 8자 천자문> 81. 광원면막 암수묘명 - 曠遠綿邈하고...,2022.05.01.,안녕하세요!\n좋은인연 모하입니다.\n오늘은\n8자씩 공부하는\n천자문\n81 번째...,19228,안녕하세요 좋은인연 모하입니다 오늘은 8자씩 공부하는 천자문 81 번째 시간입니다 ...,"[(안녕하세요 좋은인연 모하입니다, 0, 16, None, []), (오늘은 8자씩..."
19229,https://blog.naver.com/woaishan89/222693417548,[서울 이벤트]남산 벚꽃길 걷기 챌린지 도전해서 N서울타워...,2022.05.01.,안녕하세요!\n다이어트매니아 '인천이찐이야'입니다.\n'워크온'이라는 어플은 걸으면...,19229,안녕하세요 다이어트매니아 인천이찐이야 입니다 워크온 이라는 어플은 걸으면서 몇보를 ...,"[(안녕하세요 다이어트매니아 인천이찐이야 입니다, 0, 24, None, []), ..."


In [7]:
# 데이터프레임을 10개 단위로 나누기
chunk_size = 50
num_chunks = len(df) // chunk_size + 1  # 나머지가 있는 경우 추가적인 청크 필요

dfs = {}
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size
    chunk_df = df[start_idx:end_idx]
    dfs[f"df_{i+1}"] = chunk_df

# 생성된 데이터프레임들 확인
for key, chunk_df in dfs.items():
    print(key)

df_1
df_2
df_3
df_4
df_5
df_6
df_7
df_8
df_9
df_10
df_11
df_12
df_13
df_14
df_15
df_16
df_17
df_18
df_19
df_20
df_21
df_22
df_23
df_24
df_25
df_26
df_27
df_28
df_29
df_30
df_31
df_32
df_33
df_34
df_35
df_36
df_37
df_38
df_39
df_40
df_41
df_42
df_43
df_44
df_45
df_46
df_47
df_48
df_49
df_50
df_51
df_52
df_53
df_54
df_55
df_56
df_57
df_58
df_59
df_60
df_61
df_62
df_63
df_64
df_65
df_66
df_67
df_68
df_69
df_70
df_71
df_72
df_73
df_74
df_75
df_76
df_77
df_78
df_79
df_80
df_81
df_82
df_83
df_84
df_85
df_86
df_87
df_88
df_89
df_90
df_91
df_92
df_93
df_94
df_95
df_96
df_97
df_98
df_99
df_100
df_101
df_102
df_103
df_104
df_105
df_106
df_107
df_108
df_109
df_110
df_111
df_112
df_113
df_114
df_115
df_116
df_117
df_118
df_119
df_120
df_121
df_122
df_123
df_124
df_125
df_126
df_127
df_128
df_129
df_130
df_131
df_132
df_133
df_134
df_135
df_136
df_137
df_138
df_139
df_140
df_141
df_142
df_143
df_144
df_145
df_146
df_147
df_148
df_149
df_150
df_151
df_152
df_153
df_154
df_155
df_156
df_157
df_158
df

In [8]:
''' 분리된 문장 한 개를 한 row로 '''
df2 = pd.DataFrame(columns = ['split_str', 'org_idx']) #용량을 줄이기 위해 새로운 데이터프레임으로

''' 문장 데이터 프레임 저장 '''
df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_5월_전처리.csv", index=False, encoding='UTF-8')

In [9]:
import pandas as pd
df2 = pd.read_csv("../dataset/2022/본문_전처리/나무문화_본문_5월_전처리.csv", encoding='UTF-8')
df2

,split_str,org_idx


In [ ]:
for a in range(1,len(dfs)+1):
    for i in tqdm(dfs['df_%s'%a]['split_list'].index):
        for j in range(len(dfs['df_%s'%a]['split_list'][i])):
            split_str= {'split_str': dfs['df_%s'%a]['split_list'][i][j][0], 'org_idx': [i]}
            print(split_str['org_idx'])
            split_df= pd.DataFrame(split_str)
            df2 = pd.concat([df2,split_df])
            df2.reset_index(drop=True, inplace=True)
            display.clear_output(wait=True) # 출력지우기
        else:
            time.sleep(1)
df2.reset_index(drop=True, inplace=True)
df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_5월_전처리.csv", index=False, encoding='UTF-8')
df2

In [12]:
df2

,split_str,org_idx
0,아테네 이외에도 그리스는 국토 전역이 문화유산이어서 어느 지역을 가더라도 고대문화의...,0
1,나는 그중에서도 학창 시절부터 익히 들었던 펠로폰네소스 Peloponnese 라는 ...,0
2,펠로폰네소스 지역은 그리스 최초의 문화인 미케네 문명이 싹튼 지역이며 병영국가인 스...,0
3,미케네는 기원전 1500년부터 기원전 1200년까지 소아시아 이집트 키프로스를 대상...,0
4,미케네인들은 매우 전투적이어서 주변 지역을 비롯해 멀리 소아시아 지역까지 점령했다고...,0
...,...,...
498006,좋은 식사였다 밥 먹었으면 뭐다 산책이다 정말 한강 그 가로수길에서 지하로 한강 연...,11118
498007,태화강 하나로 울산 재방문 가능 그때에도 묵은 호텔 고고 나 강도 좋아하네 바다를 ...,11118
498008,담기지가 않아서 안찍었는데 우측엔 핑크빛으로 지고있었다,11118
498009,세상 초췌 동네 산책나온 주민 포스 자전거 동호회 분들이 지나가셨는데 대화가 너무 ...,11118


In [11]:
df2.isnull().sum()

split_str    0
org_idx      0
dtype: int64

In [13]:
df2.dropna(axis=0, inplace=True)
df2.reset_index(drop=True, inplace=True)
df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_5월_전처리.csv", index=False, encoding='UTF-8')